In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from preambules_list import preambule_list, Preambule
from Functions.approximations_list import Approximation
from Functions.utils import calc_mse
from Functions.approximations_list import ApproximationBase, ApproximationRestrictedLow
import pickle

stat_data = np.load('generators/generated/python_simulations.npz')

loaded_test_amplitudes = stat_data['test_amplitudes']               #[amplitude step]
loaded_all_energies = stat_data['all_energies']                     #[preamble, amplitude step, generations]
loaded_all_points = stat_data['all_points']                         #[preamble, amplitude step, generations]
loaded_mean_energies = stat_data['mean_energies']                   #[preamble, amplitude step]
loaded_mean_points = stat_data['mean_points']                       #[preamble, amplitude step]
loaded_lower_quantile_points = stat_data['lower_quantile_points']   #[preamble, amplitude step]
loaded_higher_quantile_points = stat_data['higher_quantile_points'] #[preamble, amplitude step]

In [ ]:
approximation_value_table = [
    loaded_lower_quantile_points,
    loaded_mean_points,
    loaded_higher_quantile_points
]

all_approximations = [[] for _ in range(Preambule.Last.value)]
# Add common approximations for all preamble types: low, high quantiles and mean points
for preamble_i in range(Preambule.Last.value):
    for approximation_i in range(Approximation.LastCommon.value):
        approximation_values_x = loaded_mean_energies[preamble_i]
        approximation_values_y = approximation_value_table[approximation_i][preamble_i]
        basic_approximation = ApproximationBase(Preambule(preamble_i), Approximation(approximation_i), approximation_values_x, approximation_values_y)
        all_approximations[preamble_i].append(basic_approximation)
# Add custom approximations
low_threshold = 1000
all_approximations[Preambule.ExtendedNegative.value].append(ApproximationRestrictedLow(Preambule.ExtendedNegative,
                                                                                       Approximation.RestrictedLow,
                                                                                       loaded_mean_energies[Preambule.ExtendedNegative.value],
                                                                                       approximation_value_table[Approximation.Mean.value][Preambule.ExtendedNegative.value],
                                                                                       low_threshold))


In [ ]:
# Polynomial fitting
poly_degree = 2
for preamble_approximations in all_approximations:
    for approximation in preamble_approximations:
        energy = approximation.input_x
        points = approximation.input_y
        poly_coeff = np.polyfit(energy, points, poly_degree)
        approximation.coefficients = poly_coeff

        approx_y = np.polyval(poly_coeff, energy)
        mse = calc_mse(approx_y, points)
        residuals = points - approx_y
        # Plotting approxmation and real curves

        plt.figure(figsize=(10,6))
        plt.suptitle("Approximation of " + preambule_list[approximation.preamble.value].get_name() + " preamble with "
                    + approximation.get_name() + " points")
        plt.subplot(121)
        plt.plot(energy, points, label = "Real")
        plt.plot(energy, approx_y, label = "Approximation")
        plt.title("Approximation")
        plt.xlabel("Energy")
        plt.ylabel("Correlation amplitude")
        plt.legend()
        plt.subplot(122)
        plt.plot(energy, residuals)
        plt.title("Residuals")
        plt.ylabel("Amplitude")
        plt.xlabel("Energy")
        plt.grid()
        plt.tight_layout()
        plt.show()

        print(f"MSE: {mse:.2f}")

with open('generators/generated/python_approximations.pkl', 'wb') as f:
    pickle.dump(all_approximations, f)